# Claim-based Question-Answer Hallucination Detection

Import necessary packages.

In [1]:
import time
from uqlm.longform.black_box import ClaimQAScorer
from uqlm import BlackBoxUQ

#### Load LLM and device

In [2]:
# from langchain_google_vertexai import ChatVertexAI
# llm = ChatVertexAI(model="gemini-1.5-flash")

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_type="azure",
    openai_api_version="2024-02-15-preview",
    temperature=1,  # User to set temperature
)

In [3]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using mps device


#### Setup Prompts and Black Box Scorer

In [4]:
prompts = [
    "write a paragraph about Paul McCartney",
    "write a paragraph about John Lennon"
]

In [5]:
bb_scorer = BlackBoxUQ(
    llm=llm,
    max_calls_per_min=500,  # set value to avoid rate limit error
    device=device,
    scorers=["exact_match"],
)

#### Claim-QA class

There are three methods that can be used to compute Claim-QA score.
- `generate_and_score`: If you only have prompts, call this method generate long response, decompose that response into factoids, then generate questions for each factoids, and compute question-level, factoid-level, and response-level scores.
- `score`: If you already generated long response, call this method
- `evaluate`: If you already have decomposed long responses into factoids, call this method

##### 1. `generate_and_score` method

In [6]:
start_time = time.time()
claim_qa = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
result = await claim_qa.generate_and_score(prompts=prompts)
print(f"Computation time: {time.time() - start_time} seconds")

Prompts:  ['write a paragraph about Paul McCartney', 'write a paragraph about John Lennon']
Responses:  ['Paul McCartney is a renowned British musician, singer, and songwriter, best known as a co-founder of the legendary band The Beatles, which revolutionized pop music in the 1960s. Born on June 18, 1942, in Liverpool, England, McCartney showcased his musical talent from a young age, playing various instruments and writing songs. His partnership with John Lennon formed one of the most successful songwriting duos in history, producing timeless classics like "Hey Jude," "Let It Be," and "Yesterday." After The Beatles disbanded in 1970, McCartney embarked on a successful solo career, releasing numerous albums and hits, and forming the band Wings. In addition to his music, he is an advocate for animal rights and various social causes. With numerous awards, including multiple Grammys, McCartney\'s influence on music and culture remains profound, solidifying his status as one of the most sig

In [7]:
result.to_dict()["data"]

{'prompts': ['write a paragraph about Paul McCartney',
  'write a paragraph about John Lennon'],
 'responses': ['Paul McCartney is a renowned British musician, singer, and songwriter, best known as a co-founder of the legendary band The Beatles, which revolutionized pop music in the 1960s. Born on June 18, 1942, in Liverpool, England, McCartney showcased his musical talent from a young age, playing various instruments and writing songs. His partnership with John Lennon formed one of the most successful songwriting duos in history, producing timeless classics like "Hey Jude," "Let It Be," and "Yesterday." After The Beatles disbanded in 1970, McCartney embarked on a successful solo career, releasing numerous albums and hits, and forming the band Wings. In addition to his music, he is an advocate for animal rights and various social causes. With numerous awards, including multiple Grammys, McCartney\'s influence on music and culture remains profound, solidifying his status as one of the m

In [8]:
result.to_dict()["metadata"].keys()

dict_keys(['factoids', 'response_fact_questions', 'response_fact_questions_responses', 'response_fact_questions_sampled_responses'])

#### 2. `score` method

In [9]:
claim_qa2 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
result2 = await claim_qa2.score(prompts=claim_qa.prompts, responses=claim_qa.responses)

In [12]:
result2.to_dict()["data"]

{'prompts': ['write a paragraph about Paul McCartney',
  'write a paragraph about John Lennon'],
 'responses': ['Paul McCartney is a renowned British musician, singer, and songwriter, best known as a co-founder of the legendary band The Beatles, which revolutionized pop music in the 1960s. Born on June 18, 1942, in Liverpool, England, McCartney showcased his musical talent from a young age, playing various instruments and writing songs. His partnership with John Lennon formed one of the most successful songwriting duos in history, producing timeless classics like "Hey Jude," "Let It Be," and "Yesterday." After The Beatles disbanded in 1970, McCartney embarked on a successful solo career, releasing numerous albums and hits, and forming the band Wings. In addition to his music, he is an advocate for animal rights and various social causes. With numerous awards, including multiple Grammys, McCartney\'s influence on music and culture remains profound, solidifying his status as one of the m

#### 3. `evaluate` method

In [11]:
claim_qa3 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
result3 = await claim_qa3.evaluate(prompts=claim_qa.prompts, responses=claim_qa.responses, factoids=claim_qa.factoids)

In [13]:
result3.to_dict()["data"]

{'prompts': ['write a paragraph about Paul McCartney',
  'write a paragraph about John Lennon'],
 'responses': ['Paul McCartney is a renowned British musician, singer, and songwriter, best known as a co-founder of the legendary band The Beatles, which revolutionized pop music in the 1960s. Born on June 18, 1942, in Liverpool, England, McCartney showcased his musical talent from a young age, playing various instruments and writing songs. His partnership with John Lennon formed one of the most successful songwriting duos in history, producing timeless classics like "Hey Jude," "Let It Be," and "Yesterday." After The Beatles disbanded in 1970, McCartney embarked on a successful solo career, releasing numerous albums and hits, and forming the band Wings. In addition to his music, he is an advocate for animal rights and various social causes. With numerous awards, including multiple Grammys, McCartney\'s influence on music and culture remains profound, solidifying his status as one of the m